This is a script that uses BioPython and uniprotenator to analyze our sequencing data.

In [1]:
from Bio.Seq import Seq
import string
from Bio import pairwise2

In [2]:
filename = '../sequencing data by clone/18 - Src GK/SrcGK-T7-Term.seq'

In [3]:
with open(filename, 'r') as f:
    sequence_rough = f.readlines()[1:]

In [4]:
sequence_rough

['NNNNNNNNNNNNNNNCNNTTCNNNCTTTGTTAGCAGCCGGATCTCAGTGGTGGTGGTGGTGGTGCTCGAGTGCGGCCGNN\n',
 'NGCTTGTCGACGGAGCTCGAATTCGGATCCCTATAGGTTCTCTCCAGGCTGGTACTGGGGCTCTGTCGAGGTGAAGTAGT\n',
 'CCTCCAGGAAGGCCTGCAGGTACTCAAAAGTGGGCCGCTCCTCAGGTTCCTTCCGCCAGCACTGGCACATGAGGTCATGC\n',
 'AGCGACTCGGGGCACTCGGGCGGGCAGGGCATGCGGTAGCCCCTCTCCACCTGGTCCAGCACCTCCCTGTTGACCATCCC\n',
 'TGGGTATGGCACCCGGCCCTTGGTGGTCAGCTCAGTCAGCAGGATGCCGAAGGACCAGACATCCGACTTGATGGTGAACC\n',
 'GGCCATAGAGGGCTGCCTCGGGGGCTGTCCACTTGATGGGGAACTTGGCACCTTGCCGTGCTGTGTACTCGTTGTCCTCG\n',
 'ATGAGGCGTGCCAGCCCAAAGTCAGCCACCTTGCACACCAGGTTCTCCCCCACCAGGATGTTGGCCGCCCGCAGGTCTCG\n',
 'GTGCACGTAGTTCATCCTCTCCACATAGGCCATGCCGGATGCAATCTGAGCAGCCATATCGACGAGCTGTGGCAGCCGCA\n',
 'GGTACTTGCCGGTCTCTCCCTTCAGGAAATCCAGGAGGCTCCCCTTGCTCATGTACTCAATGACGATGTAGATGGGCTCT\n',
 'TCCGACACCACTGCGTACAGCTGAACCAGCTTCTCATGCCGGANCTTCTTCATCACTTGGGCTTCCTGCAGGAAGGCCTC\n',
 'CGGGGACATGGTGCCGGGCTTCAGAGTCTTTATGGNCACTCTGGTGGTGCCGTTCCAGGTCCCCATCCAGACCTCTCCAA\n',
 'AGCAGCCCTGCCCCAGCTTCACCTCCAGCCGCAGCGACTCC

In [5]:
sequence_rough = string.join(sequence_rough)
nucleotide_sequence = sequence_rough.replace('\n ','').replace('\n','')

In [6]:
nucleotide_sequence

'NNNNNNNNNNNNNNNCNNTTCNNNCTTTGTTAGCAGCCGGATCTCAGTGGTGGTGGTGGTGGTGCTCGAGTGCGGCCGNNNGCTTGTCGACGGAGCTCGAATTCGGATCCCTATAGGTTCTCTCCAGGCTGGTACTGGGGCTCTGTCGAGGTGAAGTAGTCCTCCAGGAAGGCCTGCAGGTACTCAAAAGTGGGCCGCTCCTCAGGTTCCTTCCGCCAGCACTGGCACATGAGGTCATGCAGCGACTCGGGGCACTCGGGCGGGCAGGGCATGCGGTAGCCCCTCTCCACCTGGTCCAGCACCTCCCTGTTGACCATCCCTGGGTATGGCACCCGGCCCTTGGTGGTCAGCTCAGTCAGCAGGATGCCGAAGGACCAGACATCCGACTTGATGGTGAACCGGCCATAGAGGGCTGCCTCGGGGGCTGTCCACTTGATGGGGAACTTGGCACCTTGCCGTGCTGTGTACTCGTTGTCCTCGATGAGGCGTGCCAGCCCAAAGTCAGCCACCTTGCACACCAGGTTCTCCCCCACCAGGATGTTGGCCGCCCGCAGGTCTCGGTGCACGTAGTTCATCCTCTCCACATAGGCCATGCCGGATGCAATCTGAGCAGCCATATCGACGAGCTGTGGCAGCCGCAGGTACTTGCCGGTCTCTCCCTTCAGGAAATCCAGGAGGCTCCCCTTGCTCATGTACTCAATGACGATGTAGATGGGCTCTTCCGACACCACTGCGTACAGCTGAACCAGCTTCTCATGCCGGANCTTCTTCATCACTTGGGCTTCCTGCAGGAAGGCCTCCGGGGACATGGTGCCGGGCTTCAGAGTCTTTATGGNCACTCTGGTGGTGCCGTTCCAGGTCCCCATCCAGACCTCTCCAAAGCAGCCCTGCCCCAGCTTCACCTCCAGCCGCAGCGACTCCCGGGGGATTTCCCNACGCGTCCTTGGCGAGTCCCTGGGTCTGCATATGGNCCTNGNAAGNNAAGGNTTTCGGTCGTTG

In [7]:
seq = Seq(nucleotide_sequence)

In [8]:
print seq.reverse_complement()

ACGATNNTCCCAACGACCGAAANCCTTNNCTTNCNAGGNCCATATGCAGACCCAGGGACTCGCCAAGGACGCGTNGGGAAATCCCCCGGGAGTCGCTGCGGCTGGAGGTGAAGCTGGGGCAGGGCTGCTTTGGAGAGGTCTGGATGGGGACCTGGAACGGCACCACCAGAGTGNCCATAAAGACTCTGAAGCCCGGCACCATGTCCCCGGAGGCCTTCCTGCAGGAAGCCCAAGTGATGAAGAAGNTCCGGCATGAGAAGCTGGTTCAGCTGTACGCAGTGGTGTCGGAAGAGCCCATCTACATCGTCATTGAGTACATGAGCAAGGGGAGCCTCCTGGATTTCCTGAAGGGAGAGACCGGCAAGTACCTGCGGCTGCCACAGCTCGTCGATATGGCTGCTCAGATTGCATCCGGCATGGCCTATGTGGAGAGGATGAACTACGTGCACCGAGACCTGCGGGCGGCCAACATCCTGGTGGGGGAGAACCTGGTGTGCAAGGTGGCTGACTTTGGGCTGGCACGCCTCATCGAGGACAACGAGTACACAGCACGGCAAGGTGCCAAGTTCCCCATCAAGTGGACAGCCCCCGAGGCAGCCCTCTATGGCCGGTTCACCATCAAGTCGGATGTCTGGTCCTTCGGCATCCTGCTGACTGAGCTGACCACCAAGGGCCGGGTGCCATACCCAGGGATGGTCAACAGGGAGGTGCTGGACCAGGTGGAGAGGGGCTACCGCATGCCCTGCCCGCCCGAGTGCCCCGAGTCGCTGCATGACCTCATGTGCCAGTGCTGGCGGAAGGAACCTGAGGAGCGGCCCACTTTTGAGTACCTGCAGGCCTTCCTGGAGGACTACTTCACCTCGACAGAGCCCCAGTACCAGCCTGGAGAGAACCTATAGGGATCCGAATTCGAGCTCCGTCGACAAGCNNNCGGCCGCACTCGAGCACCACCACCACCACCACTGAGATCCGGCTGCTAACAAAGNNNGAANNGNNNNNN

In [9]:
reverse_complement = seq.reverse_complement()

In [10]:
print reverse_complement.translate()

TXXPTTEXLXXXGPYADPGTRQGRVGKSPGSRCGWR*SWGRAALERSGWGPGTAPPEXP*RL*SPAPCPRRPSCRKPK**RXSGMRSWFSCTQWCRKSPSTSSLST*ARGASWIS*RERPASTCGCHSSSIWLLRLHPAWPMWRG*TTCTETCGRPTSWWGRTWCARWLTLGWHASSRTTSTQHGKVPSSPSSGQPPRQPSMAGSPSSRMSGPSASC*LS*PPRAGCHTQGWSTGRCWTRWRGATACPARPSAPSRCMTSCASAGGRNLRSGPLLSTCRPSWRTTSPRQSPSTSLERTYRDPNSSSVDKXXAALEHHHHHH*DPAANKXXXXXXXX


/Users/hansons/anaconda/lib/python2.7/site-packages/Bio/Seq.py:1976: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning)


In [11]:
print reverse_complement[1:].translate()

RXSQRPKXXXXXXHMQTQGLAKDAXGNPPGVAAAGGEAGAGLLWRGLDGDLERHHQSXHKDSEARHHVPGGLPAGSPSDEEXPA*EAGSAVRSGVGRAHLHRH*VHEQGEPPGFPEGRDRQVPAAATARRYGCSDCIRHGLCGEDELRAPRPAGGQHPGGGEPGVQGG*LWAGTPHRGQRVHSTARCQVPHQVDSPRGSPLWPVHHQVGCLVLRHPAD*ADHQGPGAIPRDGQQGGAGPGGEGLPHALPARVPRVAA*PHVPVLAEGT*GAAHF*VPAGLPGGLLHLDRAPVPAWREPIGIRIRAPSTSXRPHSSTTTTTTEIRLLTKXEXXXXXX


In [12]:
print reverse_complement[2:].translate()

DXPNDRXPXLXRXICRPRDSPRTRXEIPRESLRLEVKLGQGCFGEVWMGTWNGTTRVXIKTLKPGTMSPEAFLQEAQVMKKXRHEKLVQLYAVVSEEPIYIVIEYMSKGSLLDFLKGETGKYLRLPQLVDMAAQIASGMAYVERMNYVHRDLRAANILVGENLVCKVADFGLARLIEDNEYTARQGAKFPIKWTAPEAALYGRFTIKSDVWSFGILLTELTTKGRVPYPGMVNREVLDQVERGYRMPCPPECPESLHDLMCQCWRKEPEERPTFEYLQAFLEDYFTSTEPQYQPGENL*GSEFELRRQAXGRTRAPPPPPLRSGC*QXXXXXXXX


In [43]:
sequencing_result = str(reverse_complement[2:].translate())

In [44]:
sequencing_result

'DXPNDRXPXLXRXICRPRDSPRTRXEIPRESLRLEVKLGQGCFGEVWMGTWNGTTRVXIKTLKPGTMSPEAFLQEAQVMKKXRHEKLVQLYAVVSEEPIYIVIEYMSKGSLLDFLKGETGKYLRLPQLVDMAAQIASGMAYVERMNYVHRDLRAANILVGENLVCKVADFGLARLIEDNEYTARQGAKFPIKWTAPEAALYGRFTIKSDVWSFGILLTELTTKGRVPYPGMVNREVLDQVERGYRMPCPPECPESLHDLMCQCWRKEPEERPTFEYLQAFLEDYFTSTEPQYQPGENL*GSEFELRRQAXGRTRAPPPPPLRSGC*QXXXXXXXX'

In [14]:
uniprotID = 'P12931'

In [15]:
from urllib2 import urlopen
from lxml import objectify

# Define url from which to download xml.
url='http://www.uniprot.org/uniprot/'
r1 = urlopen(url+'%s' % uniprotID +'.xml')

# Read xml file.
my_xml = r1.read()
# Take out annoying head that makes xml file difficult to parse.
my_xml = my_xml.replace('<uniprot xmlns="http://uniprot.org/uniprot" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://uniprot.org/uniprot http://www.uniprot.org/support/docs/uniprot.xsd">','<uniprot>', 1)

# Write xml file.
with open("%s.xml" % uniprotID,"w") as file:
     file.write(my_xml)
     
# Define list of PDBs from uniprot xml.

root = objectify.fromstring(my_xml)

In [16]:
uniprot_sequence = [element.text for element in root.findall('.//sequence')]

In [17]:
uniprot_sequence

[None,
 None,
 '\nMGSNKSKPKDASQRRRSLEPAENVHGAGGGAFPASQTPSKPASADGHRGPSAAFAPAAAE\nPKLFGGFNSSDTVTSPQRAGPLAGGVTTFVALYDYESRTETDLSFKKGERLQIVNNTEGD\nWWLAHSLSTGQTGYIPSNYVAPSDSIQAEEWYFGKITRRESERLLLNAENPRGTFLVRES\nETTKGAYCLSVSDFDNAKGLNVKHYKIRKLDSGGFYITSRTQFNSLQQLVAYYSKHADGL\nCHRLTTVCPTSKPQTQGLAKDAWEIPRESLRLEVKLGQGCFGEVWMGTWNGTTRVAIKTL\nKPGTMSPEAFLQEAQVMKKLRHEKLVQLYAVVSEEPIYIVTEYMSKGSLLDFLKGETGKY\nLRLPQLVDMAAQIASGMAYVERMNYVHRDLRAANILVGENLVCKVADFGLARLIEDNEYT\nARQGAKFPIKWTAPEAALYGRFTIKSDVWSFGILLTELTTKGRVPYPGMVNREVLDQVER\nGYRMPCPPECPESLHDLMCQCWRKEPEERPTFEYLQAFLEDYFTSTEPQYQPGENL\n']

In [18]:
uniprot_sequence[2]

'\nMGSNKSKPKDASQRRRSLEPAENVHGAGGGAFPASQTPSKPASADGHRGPSAAFAPAAAE\nPKLFGGFNSSDTVTSPQRAGPLAGGVTTFVALYDYESRTETDLSFKKGERLQIVNNTEGD\nWWLAHSLSTGQTGYIPSNYVAPSDSIQAEEWYFGKITRRESERLLLNAENPRGTFLVRES\nETTKGAYCLSVSDFDNAKGLNVKHYKIRKLDSGGFYITSRTQFNSLQQLVAYYSKHADGL\nCHRLTTVCPTSKPQTQGLAKDAWEIPRESLRLEVKLGQGCFGEVWMGTWNGTTRVAIKTL\nKPGTMSPEAFLQEAQVMKKLRHEKLVQLYAVVSEEPIYIVTEYMSKGSLLDFLKGETGKY\nLRLPQLVDMAAQIASGMAYVERMNYVHRDLRAANILVGENLVCKVADFGLARLIEDNEYT\nARQGAKFPIKWTAPEAALYGRFTIKSDVWSFGILLTELTTKGRVPYPGMVNREVLDQVER\nGYRMPCPPECPESLHDLMCQCWRKEPEERPTFEYLQAFLEDYFTSTEPQYQPGENL\n'

In [19]:
#uniprot_sequence = string.join(uniprot_sequence[2])
uniprot_sequence = uniprot_sequence[2].replace('\n','')

In [20]:
uniprot_sequence

'MGSNKSKPKDASQRRRSLEPAENVHGAGGGAFPASQTPSKPASADGHRGPSAAFAPAAAEPKLFGGFNSSDTVTSPQRAGPLAGGVTTFVALYDYESRTETDLSFKKGERLQIVNNTEGDWWLAHSLSTGQTGYIPSNYVAPSDSIQAEEWYFGKITRRESERLLLNAENPRGTFLVRESETTKGAYCLSVSDFDNAKGLNVKHYKIRKLDSGGFYITSRTQFNSLQQLVAYYSKHADGLCHRLTTVCPTSKPQTQGLAKDAWEIPRESLRLEVKLGQGCFGEVWMGTWNGTTRVAIKTLKPGTMSPEAFLQEAQVMKKLRHEKLVQLYAVVSEEPIYIVTEYMSKGSLLDFLKGETGKYLRLPQLVDMAAQIASGMAYVERMNYVHRDLRAANILVGENLVCKVADFGLARLIEDNEYTARQGAKFPIKWTAPEAALYGRFTIKSDVWSFGILLTELTTKGRVPYPGMVNREVLDQVERGYRMPCPPECPESLHDLMCQCWRKEPEERPTFEYLQAFLEDYFTSTEPQYQPGENL'

In [23]:

sequencing_result


Seq('DXPNDRXPXLXRXICRPRDSPRTRXEIPRESLRLEVKLGQGCFGEVWMGTWNGT...XXX', HasStopCodon(ExtendedIUPACProtein(), '*'))

In [45]:
alignments = pairwise2.align.globalxx(sequencing_result, uniprot_sequence)

In [46]:
alignments[0]

('---------D-------X-P--N---------------------D--R------------------------------X-P-------------------X-L----X-R-------------------------------------------X-I----C-R-------P------------------------------------R--D-------SPRT---------------------R------------------X-EIPRESLRLEVKLGQGCFGEVWMGTWNGTTRVXIKTLKPGTMSPEAFLQEAQVMKKXRHEKLVQLYAVVSEEPIYIVIEYMSKGSLLDFLKGETGKYLRLPQLVDMAAQIASGMAYVERMNYVHRDLRAANILVGENLVCKVADFGLARLIEDNEYTARQGAKFPIKWTAPEAALYGRFTIKSDVWSFGILLTELTTKGRVPYPGMVNREVLDQVERGYRMPCPPECPESLHDLMCQCWRKEPEERPTFEYLQAFLEDYFTSTEPQYQPGENL*GSEFELRRQAXGRTRAPPPPPLRSGC*QXXXXXXXX',
 'MGSNKSKPKDASQRRRSLEPAENVHGAGGGAFPASQTPSKPASADGHRGPSAAFAPAAAEPKLFGGFNSSDTVTSPQRAGPLAGGVTTFVALYDYESRTETDLSFKKGERLQIVNNTEGDWWLAHSLSTGQTGYIPSNYVAPSDSIQAEEWYFGKITRRESERLLLNAENPRGTFLVRESETTKGAYCLSVSDFDNAKGLNVKHYKIRKLDSGGFYITS-RTQFNSLQQLVAYYSKHADGLCHRLTTVCPTSKPQTQGLAKDAWEIPRESLRLEVKLGQGCFGEVWMGTWNGTTRVAIKTLKPGTMSPEAFLQEAQVMKKLRHEKLVQLYAVVSEEPIYIVTEYMSKGSLLDFLKGETGKYLRLPQLVDMAAQIASGMAYVERMNYVHRDLRAANILVGENLVCKVADFGLARLIEDNE